In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [2]:
# 数据读取
df1 = pd.read_excel('data/水土保持-重点工程.xlsx', header=1)
df1.drop(index=0, inplace=True)
df1.head(3)

,序号,省,市,县（区）,检查\n内容,项目名称,检查\n月份,管理单位,问题\n类型,问题\n描述,问题\n等级,依据监督检查办法,Unnamed: 12,Unnamed: 13,问题整改情况,备注
1,1.0,北京,北京市,门头沟,2022年度国家水土保持重点工程实施情况,小流域综合治理工程,2022-08-01 00:00:00,门头沟区水务局,前期工作,实施方案未在上一年度12月底前完成审批,一般,《水土保持工程监督检查办法（试行）》,附件,4,已完成整改,NaN
2,2.0,北京,北京市,门头沟,2022年度国家水土保持重点工程实施情况,小流域综合治理工程,2022-08-01 00:00:00,门头沟区水务局,建设管理,尚未完成招标，进度滞后,一般,《水土保持工程监督检查办法（试行）》,NaN,NaN,已完成整改,NaN
3,3.0,北京,北京市,延庆区,2021年度督查问题落实情况,小流域综合治理工程,2022-08-02 00:00:00,延庆区水务局,前期工作,未在规定时间内完成方案审批,一般,《水土保持工程监督检查办法（试行）》,附件,4,已完成整改,NaN


In [3]:
df = df1.loc[1:, ["省", "市", "县（区）", "问题\n等级"]]
df.columns = ("省", "市", "区", "级")

In [4]:
# 数据清洗

# 补全直辖县的Nan
df.loc[df.loc[df["市"].isnull()].index, "市"] = df.loc[df.loc[df["市"].isnull()].index, "区"]

In [5]:
df[df["省"]=="内蒙古"]

,省,市,区,级
46,内蒙古,呼和浩特市,和林格尔县,一般
47,内蒙古,呼和浩特市,和林格尔县,一般
48,内蒙古,呼和浩特市,和林格尔县,一般
49,内蒙古,呼和浩特市,和林格尔县,一般
50,内蒙古,呼和浩特市,和林格尔县,较重
51,内蒙古,呼和浩特市,和林格尔县,一般
52,内蒙古,呼和浩特市,和林格尔县,一般
53,内蒙古,呼和浩特市,和林格尔县,一般
54,内蒙古,呼和浩特市,和林格尔县,一般
55,内蒙古,呼和浩特市,和林格尔县,一般


In [6]:
df.groupby("省").groups['北京']

Int64Index([1, 2, 3, 4, 5, 6], dtype='int64')

In [7]:
result = {}
for province in df.groupby("省").groups:
    if province in ['北京', '天津', '上海', '重庆']:
        data = df.loc[df.groupby("省").groups[province], ["区", "级"]]
    else:
        data = df.loc[df.groupby("省").groups[province], ["市", "级"]]
    data.columns = ("检查单位", "级")
    count = data.groupby(["检查单位", "级"]).groups
    count = { key: len(value) for key, value in count.items()}
    provinceResult = {}
    for key, value in count.items():
        units = key[0].split("、")
        level = key[1]
        for unit in units:
            if unit not in provinceResult:
                provinceResult[unit] = {}
            provinceResult[unit][level] = value
    result[province] = provinceResult

In [8]:
result

{'云南': {'玉溪市': {'一般': 4}, '红河州': {'一般': 2, '较重': 1}},
 '内蒙古': {'呼和浩特市': {'一般': 17, '较重': 4}},
 '北京': {'延庆区': {'一般': 4}, '门头沟': {'一般': 2}},
 '吉林': {'白山市': {'一般': 2, '其他': 3}, '辽源市': {'一般': 2, '其他': 4}},
 '四川': {'凉山州': {'一般': 5}, '攀枝花市': {'一般': 3}, '绵阳市': {'一般': 4}},
 '宁夏': {'吴忠市': {'一般': 2, '较重': 1}, '固原市': {'一般': 4, '较重': 3}},
 '安徽': {'安庆市': {'一般': 1}, '池州市': {'一般': 1}, '宣城市': {'一般': 1}},
 '山东': {'泰安市': {'一般': 1}, '济南市': {'一般': 1}},
 '山西': {'临汾市': {'一般': 10}, '吕梁市': {'一般': 4}, '朔州市': {'一般': 16}},
 '广东': {'惠州': {'一般': 2, '较重': 1}, '梅州': {'一般': 3, '较重': 1}},
 '广西': {'柳州': {'一般': 3, '较重': 1}, '河池': {'一般': 6, '较重': 2}},
 '新疆': {'阿勒泰地区': {'一般': 4}},
 '新疆兵团': {'第十三师': {'一般': 3, '较重': 1}},
 '江苏': {'扬州市': {'一般': 1}, '南通市': {'一般': 1}},
 '江西': {'赣州市': {'一般': 8}},
 '河北': {'张家口市': {'一般': 3}, '邢台市': {'一般': 2, '较重': 1}, '邯郸市': {'一般': 3}},
 '河南': {'洛阳市': {'一般': 1, '较重': 3}},
 '海南': {'屯昌县': {'一般': 3}, '琼海市': {'一般': 3}},
 '湖北': {'十堰市': {'一般': 1}, '神农架林区': {'一般': 1, '较重': 1}},
 '湖南': {'怀化市': {'一般': 4}},

In [9]:
answer = DataFrame(columns=["省", "检查单位", "严重问题数", "较重问题数", "一般问题数"])
for province, data in result.items():
    for unit, level in data.items():
        answer.loc[len(answer)] = [province, unit, level.get("严重", 0), level.get("较重", 0), level.get("一般", 0)+level.get("其他", 0)]

In [10]:
answer.head(5)

,省,检查单位,严重问题数,较重问题数,一般问题数
0,云南,玉溪市,0,0,4
1,云南,红河州,0,1,2
2,内蒙古,呼和浩特市,0,4,17
3,北京,延庆区,0,0,4
4,北京,门头沟,0,0,2


In [11]:
answer["问题数"] = answer.apply(lambda row: row["严重问题数"] + row["较重问题数"] + row["一般问题数"], axis=1)

In [12]:
answer["量化分值"] = 10*answer["严重问题数"] + 6*answer["较重问题数"] + 2*answer["一般问题数"]

In [13]:
N = 86
answer["考核得分"] = answer["量化分值"].map(lambda x: round(100 * np.exp(-x/N), 2))


In [14]:
provinceScore = answer.groupby("省").apply(lambda group: group["考核得分"].mean())
answer["省份得分"] = answer["省"].apply(lambda province: round(provinceScore[province],2))

In [15]:
# 设置序号
answer.index = answer.index + 1
answer.head(5)

,省,检查单位,严重问题数,较重问题数,一般问题数,问题数,量化分值,考核得分,省份得分
1,云南,玉溪市,0,0,4,4,8,91.12,90.07
2,云南,红河州,0,1,2,3,10,89.02,90.07
3,内蒙古,呼和浩特市,0,4,17,21,58,50.95,50.95
4,北京,延庆区,0,0,4,4,8,91.12,93.29
5,北京,门头沟,0,0,2,2,4,95.46,93.29


In [16]:
answer.to_excel("data/水土保持-重点工程-结果.xlsx", index=True)